In [43]:
import requests
import numpy as np
import pandas as pd
from io import StringIO
from datetime import datetime
import pymongo
from pymongo import MongoClient
from dotenv import load_dotenv
import os

In [44]:
load_dotenv()

True

In [118]:
df = pd.read_json("Info.ManualFoodInfo.json")

In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         25 non-null     object
 1   address      25 non-null     object
 2   city         25 non-null     object
 3   postal_code  25 non-null     object
 4   hours        24 non-null     object
dtypes: object(5)
memory usage: 1.1+ KB


In [119]:
df.drop(["Province", "province", "days", "notes", "Phone Number", "phone_number", "_id", "provience"], axis=1, inplace=True)

In [107]:
df.replace('', np.nan, inplace=True)
df.dropna(inplace=True)

In [121]:
try:
    uri = os.getenv("MONGO_CONNECTION_STRING")
    client = MongoClient(uri)
    database = client["Info"]
    collection = database["LocationCache"]
    results = list(collection.find({}, {"_id": 0}))
    client.close()
except Exception as e:
    raise Exception(
        "The following error occurred: ", e)

In [122]:
base_geocode_url = "https://geocode.googleapis.com/v4beta/geocode/address/"
GEOCODE_MAPS_API_KEY = os.getenv("GEOCODE_MAPS_API_KEY")
try:
    uri = os.getenv("MONGO_CONNECTION_STRING")
    client = MongoClient(uri)
    database = client["Info"]
    collection = database["LocationCache"]
    for index, row in df.iterrows():
        base_address = row["address"]
        if base_address in [list(e.keys())[0] for e in results]:
            print(".", end="")
        else:
            address = str(base_address).replace(" ", "+")
            url = base_geocode_url + address + "?key=" + GEOCODE_MAPS_API_KEY
            data = requests.get(url = url).json()
            if (data):
                obj = {base_address: {
                    "name": row["name"],
                    "postal_code": row["postal_code"],
                    "city": row["city"],
                    "type": "food",
                    "lat": data["results"][0]["location"]["latitude"],
                    "long": data["results"][0]["location"]["longitude"]
                }}
                print(obj)
                results.append(obj)
                collection.insert_one(obj)
    client.close()
except Exception as e:
    raise Exception(
        "The following error occurred: ", e)

........................

In [123]:
locations = {}
for e in results:
    e_add = list(e.keys())[0]
    locations[e_add] = e[e_add]

In [124]:
try:
    uri = os.getenv("MONGO_CONNECTION_STRING")
    client = MongoClient(uri)
    database = client["Info"]
    collection = database["FoodInfo"]
    for index, row in df.iterrows():
        base_address = row["address"]
        if (base_address in list(locations.keys())):
            obj = {
                "address": base_address,
                "name": row["name"],
                "postal_code": row["postal_code"],
                "city": row["city"],
                "hours": row["hours"],
                "lat": locations[base_address]["lat"],
                "long": locations[base_address]["long"]
            }
            print(obj)
            collection.insert_one(obj)
    client.close()
        
except Exception as e:
    raise Exception(
        "The following error occurred: ", e)

{'address': '353 Sherbourne St', 'name': 'Allan Gardens Food Bank', 'postal_code': 'M5A 2S3', 'city': 'Toronto', 'hours': '12:30PM-3:00PM', 'lat': 43.663015099999996, 'long': -79.372528}
{'address': '1041 Pape Ave', 'name': 'Bethany Church Food Bank', 'postal_code': ' M4K 3W1', 'city': 'Toronto', 'hours': '1:15PM-3:00PM', 'lat': 43.689235499999995, 'long': -79.3484398}
{'address': '2305 Bloor St W', 'name': 'Bloor West Food Bank', 'postal_code': 'M6R 1L5', 'city': 'Toronto', 'hours': nan, 'lat': 43.6501668, 'long': -79.4793092}
{'address': '194 Park Lawn Rd', 'name': 'Christ Church St James Anglican Church Food Pantry', 'postal_code': 'M8Y 3J1', 'city': 'Toronto', 'hours': '1:00PM-5:30PM', 'lat': 43.6298798, 'long': -79.49115239999999}
{'address': '850 and 854 Bloor St W', 'name': 'Christie Ossington Neighbourhood Centre - Bloor Street Drop-in', 'postal_code': 'M6G 1M2', 'city': 'Toronto', 'hours': '10:00AM-4:00PM', 'lat': 43.662407699999996, 'long': -79.4242318}
{'address': '850 and 8